# canny, gaussian blur <- 이미지 노이즈 줄이기

In [2]:
import os, cv2, math
import numpy as np

In [26]:
# path = './medinode_data_001_120/001_649900540_아빌리파이정1밀리그램/000.jpg'
# 한글 경로때문에 기존 cv2.imread(path) 는 사용 불가능!!
# 그래서 해결방법은 한글 경로가 포함된 경우
# numpy 행렬로 읽은 다음 cv2.imdecode 를 통해 복호화 해주면 opencv에서 사용가능
# a = np.fromfile(path, np.uint8)

# img = cv2.imdecode(a, cv2.IMREAD_COLOR)

path = './medinode_data_001_120/001/005.jpg'

img = cv2.imread(path)

# 이미지 정보
img_w, img_h, img_c = img.shape

if img is None:
    print('경로 확인 필요')

# 윤곽선 잡기
# canny
one = 500
two = 1000
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # 외곽선 검출 그리기 cv2.drawContours
# draw = cv2.drawContours(img, contours=img_count[0][0],contourIdx=-1, color=(0,0,255), thickness=2)
# draw = cv2.drawContours(img, contours=img_count[0],contourIdx=-1, color=(0,0,255), thickness=2)

# 조건문 넣어서 여기에 박스 
# cv2.rectangle(img, 시작점, 종료점, color=색상, thickness=두께)

# cv2.imshow('org', img)
# cv2.imshow('canny', img_canny)
# cv2.imshow('gaussian', img_gaussian)
# cv2.imshow('gaussian_canny', img_gaussian_canny)
cv2.imshow('draw', draw)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# 윤곽선 검출시 총 개수
len(img_count[0])

4

In [ ]:
# 첫번째 탐지된 객체의 각 점의 좌표 <- 이거 이용해서 최대 x, y 축 찾아낼수있음 그치만 방향은 알기 어렵.
img_count[0][0]

array([[[509, 222]],

       [[508, 223]],

       [[508, 224]],

       [[507, 225]],

       [[507, 227]],

       [[509, 227]],

       [[510, 226]],

       [[510, 223]],

       [[511, 222]],

       [[510, 223]]], dtype=int32)

In [ ]:
len(img_count)

2

In [11]:
# 위에서 얻은 윤곽선 데이터를 이용해서 각 점의 x, y값 구하기
# 그러면 가장 max_x, min_x, max_y, min_y 좌표를 얻어서 탐지된 박스의 넓이 구한 후
# 넓이가 5 이하면 알약이 아니므로 그건 생략하고 나머지 객체들을 박스 지정!

# 현재는 박스 하나만 선택하고 넓이 구하고 
point_x = []
point_y = []
for i in img_count[0][0]:
    for x, y in i:
        point_x.append(x)
        point_y.append(y)

len(point_x), len(point_y)

(10, 10)

In [12]:
# x좌표
max_x = max(point_x)
min_x = min(point_x)

max_x, min_x

(511, 507)

In [13]:
# y좌표
max_y = max(point_y)
min_y = min(point_y)

max_x, min_y

(511, 222)

In [14]:
len_x = abs(max_x - min_x)
len_y = abs(max_y - min_y)

len_x, len_y

# 박스 넓이
box_area = len_x * len_y
box_area

20

In [20]:
a = list(img_count[0])

4

In [22]:
a.pop(0)

array([[[509, 222]],

       [[508, 223]],

       [[508, 224]],

       [[507, 225]],

       [[507, 227]],

       [[509, 227]],

       [[510, 226]],

       [[510, 223]],

       [[511, 222]],

       [[510, 223]]], dtype=int32)

In [23]:
import os, cv2, math

path = './medinode_data_001_120/001/004.jpg'

img = cv2.imread(path)

if img is None:
    print('경로 확인 필요')
    
# 이미지 정보
img_w, img_h, img_c = img.shape

# 윤곽선 잡기
# canny
one = 500
two = 1000
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# img_count는 윤곽선의 정보를 담고있다.
# 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
ls_count = list(img_count[0])

if len(ls_count) < 3:
    raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
else:
    drop_index = []
    index = 0
    img_count[0]
    for i in img_count[0]:
        point_x = []
        point_y = []    
        
        # 리스트 차원축소 안하면 for문 하나더 사용해야함.
        ls_2d = sum(i.tolist(), [])
        for x, y in ls_2d:
            point_x.append(x)
            point_y.append(y)
        
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(point_x)
        min_x = min(point_x)
        max_y = max(point_y)
        min_y = min(point_y)   

        # 점과 점사이의 거리 이용해서 x, y 구하기
        len_x = abs(max_x - min_x)
        len_y = abs(max_y - min_y)

        # 현재는 넓이가 50 이하면 알약이 아닌것으로 판단
        if len_x * len_y < 50:
            drop_index.append(index)
            index += 1
        else:
            index += 1

    drop_index.reverse()
    # 불필요한 객체 삭제
    for _ in drop_index:
        ls_count.pop(_)

# 박스 좌표 구하기
box_point = []
tag = 0
for ls in ls_count:
    p_x = []
    p_y = []
    ls_2d = sum(ls.tolist(), [])
    for x, y in ls_2d:
        p_x.append(x)
        p_y.append(y)
        
    # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
    max_x = max(p_x)
    min_x = min(p_x)
    max_y = max(p_y)
    min_y = min(p_y)

    # 박스의 각 좌표
    # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
    x1 = (min_x - 10, min_y - 10)
    x2 = (max_x + 10, min_y - 10)
    x3 = (min_x - 10, max_y + 10)
    x4 = (max_x + 10, max_y + 10)
    box_point.append((x1,x4))
    cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)
    pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
    window_name = f'pill {tag}'
    cv2.imshow(window_name, pill)
    tag += 1


# # 외곽선 검출 그리기 cv2.drawContours
draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

cv2.imshow('draw', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [33]:
sum((img_count[0])[0].tolist(), [])

[[509, 222],
 [508, 223],
 [508, 224],
 [507, 225],
 [507, 227],
 [509, 227],
 [510, 226],
 [510, 223],
 [511, 222],
 [510, 223]]